In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, jsonify
import pyodbc
from datetime import datetime
import json

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'  

def get_db_connection():
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=localhost;' 
        'DATABASE=Hotel_DB;' 
        'Trusted_Connection=yes;'
    )
    return conn

def row_to_dict(row):
    if row:
        return dict(zip([column[0] for column in row.cursor_description], row))
    return None

@app.route('/')
def index():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    stats = {}
    
    cursor.execute("SELECT COUNT(*) FROM Guest")
    stats['total_guests'] = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM Room WHERE Status = 'Available'")
    stats['available_rooms'] = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM Booking")
    stats['total_bookings'] = cursor.fetchone()[0]
    
    cursor.execute("SELECT SUM(Amount) FROM Payment")
    revenue = cursor.fetchone()[0]
    stats['total_revenue'] = float(revenue) if revenue else 0
    
    today = datetime.now().strftime('%Y-%m-%d')
    
    cursor.execute("""
        SELECT TOP 5 
            g.F_name + ' ' + g.L_name AS GuestName,
            r.Number AS RoomNumber,
            b.Check_In_Date,
            rt.Type_NameR AS RoomType,
            b.B_ID
        FROM Booking b
        JOIN Guest g ON b.G_ID = g.G_ID
        JOIN Room r ON b.R_ID = r.R_ID
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        WHERE b.Check_In_Date >= ?
        AND b.Status IN ('Confirmed', 'Upcoming')
        ORDER BY b.Check_In_Date
    """, today)
    upcoming_checkins = cursor.fetchall()
    
    cursor.execute("""
        SELECT TOP 5 
            g.F_name + ' ' + g.L_name AS GuestName,
            r.Number AS RoomNumber,
            b.Booking_Date,
            b.Status,
            b.B_ID
        FROM Booking b
        JOIN Guest g ON b.G_ID = g.G_ID
        JOIN Room r ON b.R_ID = r.R_ID
        ORDER BY b.Booking_Date DESC
    """)
    recent_bookings = cursor.fetchall()
    
    cursor.execute("""
        SELECT Status, COUNT(*) as Count
        FROM Room
        GROUP BY Status
        ORDER BY Status
    """)
    room_status = cursor.fetchall()
    
    conn.close()
    
    return render_template('index.html', 
                         stats=stats, 
                         upcoming_checkins=upcoming_checkins,
                         recent_bookings=recent_bookings,
                         room_status=room_status,
                         today=today)

@app.route('/guests')
def guests():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT G_ID, F_name, L_name, Email, Age, SSD
        FROM Guest
        ORDER BY L_name, F_name
    """)
    guests_data = cursor.fetchall()
    
    guests_with_phones_dict = {}
    cursor.execute("""
        SELECT G_ID, Phone 
        FROM GuestPhones 
        ORDER BY G_ID
    """)
    phones_data = cursor.fetchall()
    
    for phone_row in phones_data:
        guest_id = phone_row.G_ID
        phone = phone_row.Phone
        if guest_id not in guests_with_phones_dict:
            guests_with_phones_dict[guest_id] = []
        guests_with_phones_dict[guest_id].append(phone)
    
    guests_list = []
    for guest in guests_data:
        guest_dict = {
            'G_ID': guest.G_ID,
            'F_name': guest.F_name,
            'L_name': guest.L_name,
            'Email': guest.Email,
            'Age': guest.Age,
            'SSD': guest.SSD,
            'Phones': ', '.join(guests_with_phones_dict.get(guest.G_ID, []))
        }
        guests_list.append(guest_dict)
    
    total_guests = len(guests_list)
    
    guests_with_phones = sum(1 for guest in guests_list if guest['Phones'])
    
    guests_with_email = sum(1 for guest in guests_list if guest['Email'] and guest['Email'].strip())
    
    age_groups = []
    age_ranges = [(16, 25, '16-25'), (26, 35, '26-35'), (36, 45, '36-45'), (46, 120, '46+')]
    
    for min_age, max_age, name in age_ranges:
        count = sum(1 for guest in guests_list if min_age <= guest['Age'] <= max_age)
        percentage = (count / total_guests * 100) if total_guests > 0 else 0
        age_groups.append({
            'name': name,
            'count': count,
            'percentage': percentage
        })
    
    conn.close()
    
    return render_template('guests.html', 
                         guests=guests_list,
                         total_guests=total_guests,
                         guests_with_phones=guests_with_phones,
                         guests_with_email=guests_with_email,
                         age_groups=age_groups)

@app.route('/guest/<int:guest_id>')
def view_guest(guest_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT G_ID, F_name, L_name, Email, Age, SSD
        FROM Guest WHERE G_ID = ?
    """, guest_id)
    guest = cursor.fetchone()
    
    if not guest:
        flash('Guest not found!', 'error')
        return redirect(url_for('guests'))
    
    cursor.execute("SELECT Phone FROM GuestPhones WHERE G_ID = ?", guest_id)
    phones = [row.Phone for row in cursor.fetchall()]
    
    cursor.execute("""
        SELECT b.B_ID, r.Number, rt.Type_NameR, b.Check_In_Date, 
               b.Check_Out_Date, b.Status, b.Meal_Plan
        FROM Booking b
        JOIN Room r ON b.R_ID = r.R_ID
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        WHERE b.G_ID = ?
        ORDER BY b.Check_In_Date DESC
    """, guest_id)
    bookings = cursor.fetchall()
    
    conn.close()
    
    return render_template('view_guest.html',
                         guest=guest,
                         phones=phones,
                         bookings=bookings)

@app.route('/guest/add', methods=['GET', 'POST'])
def add_guest():
    if request.method == 'POST':
        f_name = request.form['f_name']
        l_name = request.form['l_name']
        ssd = request.form['ssd']
        email = request.form['email']
        age = request.form['age']
        phones = request.form.getlist('phones[]')
        
        conn = get_db_connection()
        cursor = conn.cursor()
        
        try:
            cursor.execute("""
                INSERT INTO Guest (F_name, L_name, SSD, Email, Age)
                VALUES (?, ?, ?, ?, ?)
            """, f_name, l_name, ssd, email, age)
            
            guest_id = cursor.execute("SELECT @@IDENTITY").fetchone()[0]
            
            for phone in phones:
                if phone.strip():
                    cursor.execute("""
                        INSERT INTO GuestPhones (G_ID, Phone)
                        VALUES (?, ?)
                    """, guest_id, phone.strip())
            
            conn.commit()
            flash('Guest added successfully!', 'success')
            return redirect(url_for('view_guest', guest_id=guest_id))
            
        except Exception as e:
            conn.rollback()
            flash(f'Error adding guest: {str(e)}', 'error')
        finally:
            conn.close()
    
    return render_template('add_guest.html')

@app.route('/guest/edit/<int:guest_id>', methods=['GET', 'POST'])
def edit_guest(guest_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    if request.method == 'POST':
        f_name = request.form['f_name']
        l_name = request.form['l_name']
        ssd = request.form['ssd']
        email = request.form['email']
        age = request.form['age']
        phones = request.form.getlist('phones[]')
        
        try:
            cursor.execute("""
                UPDATE Guest 
                SET F_name = ?, L_name = ?, SSD = ?, Email = ?, Age = ?
                WHERE G_ID = ?
            """, f_name, l_name, ssd, email, age, guest_id)
            
            cursor.execute("DELETE FROM GuestPhones WHERE G_ID = ?", guest_id)
            
            for phone in phones:
                if phone.strip():
                    cursor.execute("""
                        INSERT INTO GuestPhones (G_ID, Phone)
                        VALUES (?, ?)
                    """, guest_id, phone.strip())
            
            conn.commit()
            flash('Guest updated successfully!', 'success')
            return redirect(url_for('view_guest', guest_id=guest_id))
            
        except Exception as e:
            conn.rollback()
            flash(f'Error updating guest: {str(e)}', 'error')
        finally:
            conn.close()
    
    cursor.execute("SELECT * FROM Guest WHERE G_ID = ?", guest_id)
    guest = cursor.fetchone()
    
    cursor.execute("SELECT Phone FROM GuestPhones WHERE G_ID = ?", guest_id)
    phones = [row.Phone for row in cursor.fetchall()]
    
    conn.close()
    
    if not guest:
        flash('Guest not found!', 'error')
        return redirect(url_for('guests'))
    
    return render_template('edit_guest.html', guest=guest, phones=phones)

@app.route('/guest/delete/<int:guest_id>')
def delete_guest(guest_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT COUNT(*) FROM Booking WHERE G_ID = ?", guest_id)
        booking_count = cursor.fetchone()[0]
        
        if booking_count > 0:
            flash('Cannot delete guest with existing bookings!', 'error')
            return redirect(url_for('guests'))
        
        cursor.execute("DELETE FROM GuestPhones WHERE G_ID = ?", guest_id)
        
        cursor.execute("DELETE FROM Guest WHERE G_ID = ?", guest_id)
        
        conn.commit()
        flash('Guest deleted successfully!', 'success')
        
    except Exception as e:
        conn.rollback()
        flash(f'Error deleting guest: {str(e)}', 'error')
    finally:
        conn.close()
    
    return redirect(url_for('guests'))

@app.route('/bookings')
def bookings():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT 
            b.B_ID,
            g.F_name + ' ' + g.L_name AS GuestName,
            r.Number AS RoomNumber,
            rt.Type_NameR AS RoomType,
            b.Check_In_Date,
            b.Check_Out_Date,
            DATEDIFF(DAY, b.Check_In_Date, b.Check_Out_Date) AS StayDuration,
            b.Meal_Plan,
            b.Status,
            p.Amount,
            p.Payment_Method,
            b.G_ID,
            b.R_ID
        FROM Booking b
        JOIN Guest g ON b.G_ID = g.G_ID
        JOIN Room r ON b.R_ID = r.R_ID
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        LEFT JOIN Payment p ON b.B_ID = p.B_ID
        ORDER BY b.Check_In_Date DESC
    """)
    bookings_data = cursor.fetchall()
    
    cursor.execute("""
        SELECT r.R_ID, r.Number, rt.Type_NameR, rt.PricePerNight
        FROM Room r
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        WHERE r.Status = 'Available'
        ORDER BY r.Number
    """)
    available_rooms = cursor.fetchall()
    
    cursor.execute("SELECT G_ID, F_name, L_name FROM Guest ORDER BY L_name, F_name")
    guests = cursor.fetchall()
    
    conn.close()
    
    bookings_list = []
    for booking in bookings_data:
        bookings_list.append({
            'B_ID': booking.B_ID,
            'GuestName': booking.GuestName,
            'RoomNumber': booking.RoomNumber,
            'RoomType': booking.RoomType,
            'Check_In_Date': booking.Check_In_Date.strftime('%Y-%m-%d') if booking.Check_In_Date else 'N/A',
            'Check_Out_Date': booking.Check_Out_Date.strftime('%Y-%m-%d') if booking.Check_Out_Date else 'N/A',
            'StayDuration': booking.StayDuration,
            'Meal_Plan': booking.Meal_Plan,
            'Status': booking.Status,
            'Amount': float(booking.Amount) if booking.Amount else 0,
            'Payment_Method': booking.Payment_Method,
            'G_ID': booking.G_ID,
            'R_ID': booking.R_ID
        })
    
    total_bookings = len(bookings_list)
    confirmed_count = sum(1 for b in bookings_list if b['Status'] == 'Confirmed')
    checked_in_count = sum(1 for b in bookings_list if b['Status'] == 'Checked-in')
    completed_count = sum(1 for b in bookings_list if b['Status'] == 'Completed')
    upcoming_count = sum(1 for b in bookings_list if b['Status'] == 'Upcoming')
    
    meal_stats = {}
    for booking in bookings_list:
        meal = booking['Meal_Plan']
        if meal not in meal_stats:
            meal_stats[meal] = {'count': 0, 'percentage': 0}
        meal_stats[meal]['count'] += 1
    
    for meal in meal_stats:
        meal_stats[meal]['percentage'] = (meal_stats[meal]['count'] / total_bookings * 100) if total_bookings > 0 else 0
    
    return render_template('bookings.html', 
                         bookings=bookings_list,
                         total_bookings=total_bookings,
                         confirmed_count=confirmed_count,
                         checked_in_count=checked_in_count,
                         completed_count=completed_count,
                         upcoming_count=upcoming_count,
                         meal_stats=meal_stats,
                         available_rooms=available_rooms,
                         guests=guests)

@app.route('/booking/view/<int:booking_id>')
def view_booking(booking_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT 
            b.B_ID,
            g.F_name + ' ' + g.L_name AS GuestName,
            g.G_ID,
            r.Number AS RoomNumber,
            r.R_ID,
            rt.Type_NameR AS RoomType,
            rt.PricePerNight,
            b.Check_In_Date,
            b.Check_Out_Date,
            DATEDIFF(DAY, b.Check_In_Date, b.Check_Out_Date) AS StayDuration,
            b.Meal_Plan,
            b.Status,
            b.Booking_Date,
            p.Amount,
            p.Payment_Method,
            p.Payment_Date,
            (rt.PricePerNight * DATEDIFF(DAY, b.Check_In_Date, b.Check_Out_Date)) AS CalculatedTotal
        FROM Booking b
        JOIN Guest g ON b.G_ID = g.G_ID
        JOIN Room r ON b.R_ID = r.R_ID
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        LEFT JOIN Payment p ON b.B_ID = p.B_ID
        WHERE b.B_ID = ?
    """, booking_id)
    
    booking = cursor.fetchone()
    
    if not booking:
        flash('Booking not found!', 'error')
        return redirect(url_for('bookings'))
    
    conn.close()
    
    return render_template('view_booking.html', booking=booking)

@app.route('/booking/add', methods=['POST'])
def add_booking():
    guest_id = request.form['guest_id']
    room_id = request.form['room_id']
    check_in_date = request.form['check_in_date']
    check_out_date = request.form['check_out_date']
    meal_plan = request.form['meal_plan']
    booking_date = datetime.now().strftime('%Y-%m-%d')
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT Status FROM Room WHERE R_ID = ?", room_id)
        room_status = cursor.fetchone()
        
        if not room_status or room_status[0] != 'Available':
            flash('Selected room is not available!', 'error')
            return redirect(url_for('bookings'))
        
        cursor.execute("""
            INSERT INTO Booking (G_ID, R_ID, Check_In_Date, Check_Out_Date, 
                                Booking_Date, Meal_Plan, Status)
            VALUES (?, ?, ?, ?, ?, ?, 'Confirmed')
        """, guest_id, room_id, check_in_date, check_out_date, booking_date, meal_plan)
        
        booking_id = cursor.execute("SELECT @@IDENTITY").fetchone()[0]
        
        cursor.execute("UPDATE Room SET Status = 'Occupied' WHERE R_ID = ?", room_id)
        
        conn.commit()
        flash('Booking created successfully!', 'success')
        return redirect(url_for('view_booking', booking_id=booking_id))
        
    except Exception as e:
        conn.rollback()
        flash(f'Error creating booking: {str(e)}', 'error')
        return redirect(url_for('bookings'))
    finally:
        conn.close()

@app.route('/booking/edit/<int:booking_id>', methods=['GET', 'POST'])
def edit_booking(booking_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    if request.method == 'POST':
        meal_plan = request.form['meal_plan']
        status = request.form['status']
        check_in_date = request.form['check_in_date']
        check_out_date = request.form['check_out_date']
        
        try:
            cursor.execute("""
                UPDATE Booking 
                SET Meal_Plan = ?, Status = ?, Check_In_Date = ?, Check_Out_Date = ?
                WHERE B_ID = ?
            """, meal_plan, status, check_in_date, check_out_date, booking_id)
            
            if status == 'Completed':
                cursor.execute("""
                    UPDATE Room r
                    SET Status = 'Available'
                    FROM Room r
                    JOIN Booking b ON r.R_ID = b.R_ID
                    WHERE b.B_ID = ?
                """, booking_id)
            
            conn.commit()
            flash('Booking updated successfully!', 'success')
            return redirect(url_for('view_booking', booking_id=booking_id))
            
        except Exception as e:
            conn.rollback()
            flash(f'Error updating booking: {str(e)}', 'error')
        finally:
            conn.close()
    
    cursor.execute("""
        SELECT b.*, g.F_name + ' ' + g.L_name AS GuestName, r.Number AS RoomNumber
        FROM Booking b
        JOIN Guest g ON b.G_ID = g.G_ID
        JOIN Room r ON b.R_ID = r.R_ID
        WHERE b.B_ID = ?
    """, booking_id)
    
    booking = cursor.fetchone()
    
    if not booking:
        flash('Booking not found!', 'error')
        return redirect(url_for('bookings'))
    
    conn.close()
    
    return render_template('edit_booking.html', booking=booking)

@app.route('/booking/checkin/<int:booking_id>')
def checkin_booking(booking_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("""
            UPDATE Booking 
            SET Status = 'Checked-in'
            WHERE B_ID = ? AND Status = 'Confirmed'
        """, booking_id)
        
        if cursor.rowcount == 0:
            flash('Booking cannot be checked in (already checked in or not confirmed)!', 'error')
        else:
            conn.commit()
            flash('Booking checked in successfully!', 'success')
            
    except Exception as e:
        conn.rollback()
        flash(f'Error checking in booking: {str(e)}', 'error')
    finally:
        conn.close()
    
    return redirect(url_for('bookings'))

@app.route('/booking/delete/<int:booking_id>')
def delete_booking(booking_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT R_ID FROM Booking WHERE B_ID = ?", booking_id)
        room_id = cursor.fetchone()
        
        cursor.execute("DELETE FROM Payment WHERE B_ID = ?", booking_id)
        
        cursor.execute("DELETE FROM Booking WHERE B_ID = ?", booking_id)
        
        if room_id:
            cursor.execute("UPDATE Room SET Status = 'Available' WHERE R_ID = ?", room_id[0])
        
        conn.commit()
        flash('Booking deleted successfully!', 'success')
        
    except Exception as e:
        conn.rollback()
        flash(f'Error deleting booking: {str(e)}', 'error')
    finally:
        conn.close()
    
    return redirect(url_for('bookings'))

@app.route('/booking/add_payment/<int:booking_id>', methods=['POST'])
def add_payment(booking_id):
    amount = request.form['amount']
    payment_method = request.form['payment_method']
    payment_date = datetime.now().strftime('%Y-%m-%d')
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("""
            INSERT INTO Payment (B_ID, Amount, Payment_Date, Payment_Method)
            VALUES (?, ?, ?, ?)
        """, booking_id, amount, payment_date, payment_method)
        
        conn.commit()
        flash('Payment added successfully!', 'success')
        
    except Exception as e:
        conn.rollback()
        flash(f'Error adding payment: {str(e)}', 'error')
    finally:
        conn.close()
    
    return redirect(url_for('view_booking', booking_id=booking_id))

@app.route('/rooms')
def rooms():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT 
            r.R_ID,
            r.Number,
            r.Floor_Hotel,
            r.Status,
            rt.Type_NameR AS RoomType,
            rt.PricePerNight,
            s.F_name + ' ' + s.L_name AS StaffName,
            s.Role
        FROM Room r
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        JOIN Staff_Shift s ON r.Staff_ID_FK = s.StaffShift_ID
        ORDER BY r.Number
    """)
    rooms_data = cursor.fetchall()
    
    cursor.execute("SELECT StaffShift_ID, F_name, L_name, Role FROM Staff_Shift ORDER BY L_name, F_name")
    staff_list = cursor.fetchall()
    
    cursor.execute("SELECT RoomType_ID, Type_NameR, PricePerNight FROM RoomType ORDER BY Type_NameR")
    room_types = cursor.fetchall()
    
    conn.close()
    
    rooms_list = []
    for room in rooms_data:
        rooms_list.append({
            'R_ID': room.R_ID,
            'Number': room.Number,
            'Floor_Hotel': room.Floor_Hotel,
            'Status': room.Status,
            'RoomType': room.RoomType,
            'PricePerNight': float(room.PricePerNight) if room.PricePerNight else 0,
            'StaffName': room.StaffName,
            'Role': room.Role
        })
    
    total_rooms = len(rooms_list)
    available_count = sum(1 for r in rooms_list if r['Status'] == 'Available')
    occupied_count = sum(1 for r in rooms_list if r['Status'] == 'Occupied')
    cleaning_count = sum(1 for r in rooms_list if r['Status'] == 'Cleaning')
    
    floor_data = {}
    floors = set()
    for room in rooms_list:
        floor = room['Floor_Hotel']
        floors.add(floor)
        if floor not in floor_data:
            floor_data[floor] = []
        floor_data[floor].append(room)
    
    room_type_stats = {}
    for room in rooms_list:
        room_type = room['RoomType']
        if room_type not in room_type_stats:
            room_type_stats[room_type] = {
                'count': 0,
                'total_price': 0,
                'available': 0
            }
        room_type_stats[room_type]['count'] += 1
        room_type_stats[room_type]['total_price'] += room['PricePerNight']
        if room['Status'] == 'Available':
            room_type_stats[room_type]['available'] += 1
    
    for room_type in room_type_stats:
        room_type_stats[room_type]['avg_price'] = (
            room_type_stats[room_type]['total_price'] / room_type_stats[room_type]['count']
        )
    
    return render_template('rooms.html', 
                         rooms=rooms_list,
                         total_rooms=total_rooms,
                         available_count=available_count,
                         occupied_count=occupied_count,
                         cleaning_count=cleaning_count,
                         floor_data=floor_data,
                         floors=sorted(list(floors)),
                         room_type_stats=room_type_stats,
                         staff_list=staff_list,
                         room_types=room_types)

@app.route('/room/add', methods=['POST'])
def add_room():
    number = request.form['number']
    floor_hotel = request.form['floor_hotel']
    roomtype_id = request.form['roomtype_id']
    staff_id = request.form['staff_id']
    status = 'Available' 
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("""
            INSERT INTO Room (Number, Floor_Hotel, Status, RoomType_ID, Staff_ID_FK)
            VALUES (?, ?, ?, ?, ?)
        """, number, floor_hotel, status, roomtype_id, staff_id)
        
        conn.commit()
        flash('Room added successfully!', 'success')
        
    except Exception as e:
        conn.rollback()
        flash(f'Error adding room: {str(e)}', 'error')
    finally:
        conn.close()
    
    return redirect(url_for('rooms'))

@app.route('/room/update_status/<int:room_id>', methods=['POST'])
def update_room_status(room_id):
    status = request.form['status']
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("""
            UPDATE Room SET Status = ? WHERE R_ID = ?
        """, status, room_id)
        
        conn.commit()
        flash(f'Room status updated to {status}!', 'success')
        
    except Exception as e:
        conn.rollback()
        flash(f'Error updating room status: {str(e)}', 'error')
    finally:
        conn.close()
    
    return redirect(url_for('rooms'))

@app.route('/room/delete/<int:room_id>')
def delete_room(room_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT COUNT(*) FROM Booking WHERE R_ID = ?", room_id)
        booking_count = cursor.fetchone()[0]
        
        if booking_count > 0:
            flash('Cannot delete room with existing bookings!', 'error')
            return redirect(url_for('rooms'))
        
        cursor.execute("DELETE FROM Room WHERE R_ID = ?", room_id)
        conn.commit()
        flash('Room deleted successfully!', 'success')
        
    except Exception as e:
        conn.rollback()
        flash(f'Error deleting room: {str(e)}', 'error')
    finally:
        conn.close()
    
    return redirect(url_for('rooms'))

@app.route('/reports')
def reports():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT 
            rt.Type_NameR AS Room_Type,
            SUM(p.Amount) AS Total_Revenue
        FROM Payment p
        JOIN Booking b ON p.B_ID = b.B_ID
        JOIN Room r ON b.R_ID = r.R_ID
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        GROUP BY rt.Type_NameR
        ORDER BY Total_Revenue DESC
    """)
    revenue_by_type = cursor.fetchall()
    
    cursor.execute("""
        SELECT 
            YEAR(b.Check_In_Date) AS Year,
            MONTH(b.Check_In_Date) AS Month,
            COUNT(b.B_ID) AS NumberOfBookings,
            SUM(rt.PricePerNight * DATEDIFF(DAY, b.Check_In_Date, b.Check_Out_Date)) AS TotalRevenue
        FROM Booking b
        JOIN Room r ON b.R_ID = r.R_ID
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        GROUP BY YEAR(b.Check_In_Date), MONTH(b.Check_In_Date)
        ORDER BY Year DESC, Month DESC
    """)
    monthly_stats = cursor.fetchall()
    
    cursor.execute("""
        SELECT TOP 5 
            g.F_name + ' ' + g.L_name AS GuestName,
            SUM(p.Amount) AS TotalPaid,
            COUNT(b.B_ID) AS TotalBookings
        FROM Payment p
        JOIN Booking b ON p.B_ID = b.B_ID
        JOIN Guest g ON b.G_ID = g.G_ID
        GROUP BY g.F_name, g.L_name
        ORDER BY TotalPaid DESC
    """)
    top_guests = cursor.fetchall()
    
    revenue_list = []
    for rev in revenue_by_type:
        revenue_list.append({
            'Room_Type': rev.Room_Type,
            'Total_Revenue': float(rev.Total_Revenue) if rev.Total_Revenue else 0
        })
    
    monthly_list = []
    for month in monthly_stats:
        monthly_list.append({
            'Year': month.Year,
            'Month': month.Month,
            'NumberOfBookings': month.NumberOfBookings,
            'TotalRevenue': float(month.TotalRevenue) if month.TotalRevenue else 0
        })
    
    top_guests_list = []
    for guest in top_guests:
        top_guests_list.append({
            'GuestName': guest.GuestName,
            'TotalPaid': float(guest.TotalPaid) if guest.TotalPaid else 0,
            'TotalBookings': guest.TotalBookings
        })
    
    total_revenue = sum(rev['Total_Revenue'] for rev in revenue_list)
    
    total_bookings_from_stats = sum(month['NumberOfBookings'] for month in monthly_list)
    
    now = datetime.now()
    
    conn.close()
    
    return render_template('reports.html', 
                         revenue_by_type=revenue_list,
                         monthly_stats=monthly_list,
                         top_guests=top_guests_list,
                         total_revenue=total_revenue,
                         total_bookings=total_bookings_from_stats,
                         now=now)

@app.route('/api/available_rooms')
def api_available_rooms():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT r.R_ID, r.Number, rt.Type_NameR, rt.PricePerNight
        FROM Room r
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        WHERE r.Status = 'Available'
        ORDER BY r.Number
    """)
    
    rooms = cursor.fetchall()
    conn.close()
    
    rooms_list = []
    for room in rooms:
        rooms_list.append({
            'R_ID': room.R_ID,
            'Number': room.Number,
            'Type_NameR': room.Type_NameR,
            'PricePerNight': float(room.PricePerNight) if room.PricePerNight else 0
        })
    
    return jsonify(rooms_list)

@app.route('/api/room_details/<int:room_id>')
def api_room_details(room_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT r.*, rt.Type_NameR, rt.PricePerNight,
               s.F_name + ' ' + s.L_name AS StaffName
        FROM Room r
        JOIN RoomType rt ON r.RoomType_ID = rt.RoomType_ID
        JOIN Staff_Shift s ON r.Staff_ID_FK = s.StaffShift_ID
        WHERE r.R_ID = ?
    """, room_id)
    
    room = cursor.fetchone()
    conn.close()
    
    if room:
        room_dict = dict(zip([column[0] for column in cursor.description], room))
        return jsonify(room_dict)
    
    return jsonify({'error': 'Room not found'}), 404

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

@app.errorhandler(500)
def internal_server_error(e):
    return render_template('500.html'), 500

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Dec/2025 11:50:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2025 11:50:14] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Dec/2025 11:50:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2025 11:50:18] "GET /guests HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2025 11:50:18] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Dec/2025 11:50:19] "GET /bookings HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2025 11:50:19] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Dec/2025 11:50:19] "GET /api/available_rooms HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2025 11:50:20] "GET /rooms HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2025 11:50:20] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Dec/2025 11:50:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2025 11:50:22] "GET /static/style.css HTTP/1.1" 304 -
